In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# for better plots
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

import scipy.stats as stats
from sklearn.preprocessing import scale
from sklearn.cross_validation import KFold
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.ensemble import GradientBoostingClassifier as GBM

In [32]:
secom = pd.read_table('secom.data', header=None, delim_whitespace=True)
Y = pd.read_table('secom_labels.data', header=None, usecols=[0], squeeze=True, delim_whitespace=True)

print('Temos {} linhas/exemplos e {} colunas/variáveis.'\
      .format(secom.shape[0], secom.shape[1]))
print('A classe majoritária possui {} exemplos e a minoritária {}.'\
      .format(Y[Y == -1].size, Y[Y == 1].size))

Temos 1567 linhas/exemplos e 590 colunas/variáveis.
A classe majoritária possui 1463 exemplos e a minoritária 104.


In [33]:
# dados representados com numpy array
X = secom.as_matrix(columns=None)
labels = Y.as_matrix(columns=None)

# Preprocessamento
### Substitua os dados faltantes pela media da coluna (imputação pela média)

In [34]:
# calculando a média das colunas
col_mean = stats.nanmean(X, axis=0)

#ind obtém os índices onde a coluna possui dados faltantes
ind = np.where(np.isnan(X))

# substitui os índices faltantes pela média da coluna
X[ind] = np.take(col_mean, ind[1])
print(X.shape)


(1567, 590)


In [90]:
### Mini teste com uma matriz pequena para visualizar se estou fazendo a substituição das colunas pela média

import scipy.stats as stats
A = np.array([[1.0     , 2.0, np.nan],
             [np.nan, 3.0, np.nan],
             [3.0     , np.nan, 5.0]])
print('Matriz teste com colunas nan:\n', A)
col_mean = stats.nanmean(A, axis=0)
print()
for i in range(3):
    print('A média da %d coluna é: %s'% (i+1,col_mean[i]))

inds = np.where(np.isnan(A))
A[inds] = np.take(col_mean, inds[1])
print('\nMatriz substituída com as médias:\n', A)


Matriz teste com colunas nan:
 [[  1.   2.  nan]
 [ nan   3.  nan]
 [  3.  nan   5.]]

A média da 1 coluna é: 2.0
A média da 2 coluna é: 2.5
A média da 3 coluna é: 5.0

Matriz substituída com as médias:
 [[ 1.   2.   5. ]
 [ 2.   3.   5. ]
 [ 3.   2.5  5. ]]


### Finalmente padronize as colunas para media 0 e desvio padrao 1.

In [36]:
# scale() fornece média zero das colunas e desvio padrão = 1
std_secom = scale(X)

# PCA

### Vamos encontrar 80% da variância e transformar os Dados. 


In [37]:
# função PCA com sklearn
def doPCA(x): # x é o argumento do número de componentes do PCA
    from sklearn.decomposition import PCA
    pca = PCA(n_components=x)
    pca.fit(std_secom)
    return pca



In [38]:
# executando o PCA com 590 componentes.
pca = doPCA(std_secom.shape[1]) 

# os elementos em _var contém a total variância.
_var = pca.explained_variance_ratio_ 

In [71]:
# armazena a variância.
requested_var = 0

# 80%  é a variância requisitada.
lim = 0.80

# iterações para encontrar a quantidade de dimensões que mantém
# 80% da variância
for i in range (len(_var)):
    requested_var += _var[i]
    if requested_var >= lim:
        element = i
        requested_var -= _var[i] # para ajuste do indice
        break

# repetindo o PCA com 80% da variância
pca = doPCA(element) # element == 89 componentes
_var = pca.explained_variance_ratio_



### Para 80% da variância é necessário manter 84 elementos. 

#### Vamos transformar os dados para fazer o KNN.

In [72]:
# dados transformados
secom_PCA_80 = pca.transform(std_secom)

# prints
print('Com %d elementos temos a variância requisitada.' % (element))
print('Formato dos dados com PCA (variância=80%): {}.'\
      .format(secom_PCA_80.shape))
print('As %d componentes/colunas principais:' % (element))
np.set_printoptions(precision=3)
print(_var)
print('\nA soma dessas componentes/colunas: %.5f.' % np.sum(_var))


Com 89 elementos temos a variância requisitada.
Formato dos dados com PCA (variância=80%): (1567, 89).
As 89 componentes/colunas principais:
[ 0.056  0.036  0.028  0.025  0.022  0.021  0.02   0.018  0.018  0.016
  0.015  0.013  0.013  0.013  0.013  0.012  0.011  0.011  0.011  0.011
  0.01   0.01   0.01   0.01   0.01   0.009  0.009  0.009  0.009  0.008
  0.008  0.008  0.008  0.008  0.008  0.008  0.008  0.007  0.007  0.007
  0.007  0.007  0.007  0.007  0.007  0.007  0.006  0.006  0.006  0.006
  0.006  0.006  0.006  0.006  0.006  0.006  0.006  0.005  0.005  0.005
  0.005  0.005  0.005  0.005  0.005  0.005  0.005  0.005  0.005  0.004
  0.004  0.004  0.004  0.004  0.004  0.004  0.004  0.004  0.004  0.004
  0.004  0.004  0.004  0.003  0.003  0.003  0.003  0.003  0.003]

A soma dessas componentes/colunas: 0.79580.


# KNN com 5-fold externo e 3-fold interno

In [60]:
# acurácia inicial
acc_knn = 0

# separa os dados transformados  secom_PCA_80 em 5-fold
kf_5 = KFold(n = len(labels), n_folds=5)

#contador dos folds
count = 0

# para cada fold externo encontrar o melhor hiperparâmetro
for train, test in kf_5:
    count += 1
    kf_3 = KFold(n = len(labels[train]), n_folds=3)
    
    # faz um grid search do melhor k em cada conjunto de treino do 5-fold
    best_k, best_acc = 0,0
        
    for k in [1, 5, 11, 15, 21, 25]:
        new_acc = 0
        
        for train_hp, test_hp in kf_3:
            # define o knn            
            knn = KNN(n_neighbors=k)
            # treina secom_PCA_80
            knn.fit(secom_PCA_80[train][train_hp], labels[train][train_hp]) 
            # avalia a acurária 
            new_acc += knn.score(secom_PCA_80[train][test_hp], labels[train][test_hp])
        
        # acurácia média do hiperparâmetro k
        new_acc = new_acc/3
        
        # verifica se acurácia do hiperparametro k atual é a melhor
        if (new_acc > best_acc):
            best_acc = new_acc
            best_k = k
    
    # imprime a melhor acurácia obtida pelo fold interno
    print('Em fold #{}: o melhor k é: {}'.format(count, best_k))
    
    # ao fim do loop, calcula a acurácia para o melhor k
    knn_out = KNN(n_neighbors=best_k)
    
    # treina com os valores do 5-fold externo
    knn_out.fit(secom_PCA_80[train], labels[train]) 
    
    # obtém a acurácia usando o conjunto de teste do 5-fold externo
    print('Acurácia do Fold = {}%'.format(100*knn_out.score(secom_PCA_80[test],\
           labels[test])))
   
    # acha a acurácia média do método
    acc_knn += knn_out.score(secom_PCA_80[test], labels[test])

acc_knn /= 5    
print('\nAcurácia do Método: {}%'.format(acc_knn*100))


Em fold #1: o melhor k é: 11
Acurácia do Fold = 85.98726114649682%
Em fold #2: o melhor k é: 11
Acurácia do Fold = 93.31210191082803%
Em fold #3: o melhor k é: 11
Acurácia do Fold = 96.48562300319489%
Em fold #4: o melhor k é: 11
Acurácia do Fold = 96.48562300319489%
Em fold #5: o melhor k é: 11
Acurácia do Fold = 94.56869009584665%

Acurácia do Método: 93.36785983191224%


# SVM
### Para o SVM RBF teste para C=2**(-5), 2**(0), 2**(5), 2**(10) e gamma= 2**(-15) 2**(-10) 2**(-5) 2**(0) 2**(5).

In [41]:
std_secom.shape

(1567, 590)

In [62]:
# acurácia inicial
acc_svm = 0

# separa os dados transformados  std_secom em 5-fold
kf_5 = KFold(n = len(labels), n_folds=5)

#contador dos folds
count = 0

# para cada fold externo encontrar o melhor hiperparâmetro
for train, test in kf_5:
    count += 1
    kf_3 = KFold(n = len(labels[train]), n_folds=3)
    
    # faz um grid search do melhor k em cada conjunto de treino do 5-fold
    best_c, best_gamma, best_acc = 0,0,0
        
    for c in [2**-5, 2**0, 2**5, 2**10]:
        for gamma in [2**-15, 2**-10, 2**-5, 2**0, 2**5]:
            new_acc = 0
            for train_hp, test_hp in kf_3:
                # define o SVM classificador svc
                svc = SVC(C=c, kernel='rbf', gamma=gamma)
                # treina std_secom
                svc.fit(std_secom[train][train_hp], labels[train][train_hp]) 
                # avalia a acurária 
                new_acc += svc.score(std_secom[train][test_hp], \
                                     labels[train][test_hp])
        
            # acurácia média do hiperparâmetro k
            new_acc = new_acc/3
        
            # verifica se acurácia do hiperparametro k atual é a melhor
            if (new_acc > best_acc):
                best_acc = new_acc
                best_c = c
                best_gamma = gamma
    
    # imprime a melhor acurácia obtida pelo fold interno
    print('Em fold #%d: o melhor c é: %.6f e gamma: %.6f'
           % (count, best_c, best_gamma))
    
    # ao fim do loop, calcula a acurácia para o melhor c e gamma
    svc_out = SVC(C=best_c, kernel='rbf', gamma=best_gamma)
    
    # treina com os valores do 5-fold externo
    svc_out.fit(std_secom[train], labels[train]) 
    
    # obtém a acurácia usando o conjunto de teste do 5-fold externo
    print('Acurácia do Fold = {}%'.format(100*svc_out.score(std_secom[test],\
           labels[test])))
   
    # acha a acurácia média do método
    acc_svm += svc_out.score(std_secom[test], labels[test])

acc_svm /= 5    
print('\nAcurácia do Método: {}%'.format(acc_svm*100))


Em fold #1: o melhor c é: 0.031250 e gamma: 0.000031
Acurácia do Fold = 85.98726114649682%
Em fold #2: o melhor c é: 0.031250 e gamma: 0.000031
Acurácia do Fold = 93.31210191082803%
Em fold #3: o melhor c é: 0.031250 e gamma: 0.000031
Acurácia do Fold = 96.48562300319489%
Em fold #4: o melhor c é: 0.031250 e gamma: 0.000031
Acurácia do Fold = 96.48562300319489%
Em fold #5: o melhor c é: 0.031250 e gamma: 0.000031
Acurácia do Fold = 94.56869009584665%

Acurácia do Método: 93.36785983191224%


# Rede Neural
###  Para a rede neural, teste com 10, 20, 30 e 40 neuronios na camada escondida.

In [43]:
# acurácia inicial
acc = 0

# separa os dados transformados  std_secom em 5-fold
kf_5 = KFold(n = len(labels), n_folds=5)

#contador dos folds
count = 0

# para cada fold externo encontrar o melhor hiperparâmetro
for train, test in kf_5:
    count += 1
    kf_3 = KFold(n = len(labels[train]), n_folds=3)
    
    # faz um grid search do melhor k em cada conjunto de treino do 5-fold
    best_neur, best_acc = 0,0
        
    for neur in [10, 20, 30, 40]:
        new_acc = 0
        
        for train_hp, test_hp in kf_3:
            # define o knn            
            nn = MLP(solver='lbfgs', alpha=1, hidden_layer_sizes=neur, random_state=1)
            # treina std_secom
            nn.fit(std_secom[train][train_hp], labels[train][train_hp]) 
            # avalia a acurária 
            new_acc += nn.score(std_secom[train][test_hp], labels[train][test_hp])
        
        # acurácia média do hiperparâmetro k
        new_acc = new_acc/3
        
        # verifica se acurácia do hiperparametro k atual é a melhor
        if (new_acc > best_acc):
            best_acc = new_acc
            best_neur = neur
    
    # imprime a melhor acurácia obtida pelo fold interno
    print('Em fold #{}: o melhor número de neurônios na camada escondida é: {}'.format(count, best_neur))
    
    # ao fim do loop, calcula a acurácia para o melhor k
    nn_out = MLP(solver='lbfgs', alpha=1, hidden_layer_sizes=best_neur, random_state=1)
    
    # treina com os valores do 5-fold externo
    nn_out.fit(std_secom[train], labels[train]) 
    
    # obtém a acurácia usando o conjunto de teste do 5-fold externo
    print('Acurácia do Fold = {}%'.format(100*nn_out.score(std_secom[test],\
           labels[test])))
   
    # acha a acurácia média do método
    acc += nn_out.score(std_secom[test], labels[test])

acc /= 5    
print('\nAcurácia do Método: {}%'.format(acc*100))


Em fold #1: o melhor número de neurônios na camada escondida é: 20
Acurácia do Fold = 76.75159235668791%
Em fold #2: o melhor número de neurônios na camada escondida é: 40
Acurácia do Fold = 90.76433121019109%
Em fold #3: o melhor número de neurônios na camada escondida é: 30
Acurácia do Fold = 94.56869009584665%
Em fold #4: o melhor número de neurônios na camada escondida é: 30
Acurácia do Fold = 94.56869009584665%
Em fold #5: o melhor número de neurônios na camada escondida é: 40
Acurácia do Fold = 93.9297124600639%

Acurácia do Método: 90.11660324372724%


# Random Forest 

### Para o RF, teste com n_featrues = 10, 15, 20, 25 e ntrees = 100, 200, 300 e 400

In [50]:
# acurácia inicial
acc = 0

# separa os dados transformados  std_secom em 5-fold
kf_5 = KFold(n = len(labels), n_folds=5)

#contador dos folds
count = 0

# para cada fold externo encontrar o melhor hiperparâmetro
for train, test in kf_5:
    count += 1
    kf_3 = KFold(n = len(labels[train]), n_folds=3)
    
    # faz um grid search do melhor k em cada conjunto de treino do 5-fold
    best_n_features, best_n_features, best_acc = 0,0,0
        
    for n_features in [10, 15, 20, 25]:
        for n_trees in [100, 200, 300, 400]:
            new_acc = 0
            for train_hp, test_hp in kf_3:
                # define o SVM classificador svc
                rf = RF(max_depth=n_trees, n_estimators=n_features)
                # treina std_secom
                rf.fit(std_secom[train][train_hp], labels[train][train_hp]) 
                # avalia a acurária 
                new_acc += rf.score(std_secom[train][test_hp], \
                                     labels[train][test_hp])
        
            # acurácia média do hiperparâmetro k
            new_acc = new_acc/3
        
            # verifica se acurácia do hiperparametro k atual é a melhor
            if (new_acc > best_acc):
                best_acc        = new_acc
                best_n_features = n_features
                best_n_trees    = n_trees
    
    # imprime a melhor acurácia obtida pelo fold interno
    print('Em fold #%d: o melhor n_feature é: %d e n_tree: %d'
           % (count, best_n_features, best_n_trees))
    
    # ao fim do loop, calcula a acurácia para o melhor c e gamma
    rf_out = RF(max_features=best_n_features, max_depth=best_n_trees)
    
    # treina com os valores do 5-fold externo
    rf_out.fit(std_secom[train], labels[train]) 
    
    # obtém a acurácia usando o conjunto de teste do 5-fold externo
    print('Acurácia do Fold = {}%'.format(100*rf_out.score(std_secom[test],\
           labels[test])))
   
    # acha a acurácia média do método
    acc += rf_out.score(std_secom[test], labels[test])

acc /= 5    
print('\nAcurácia do Método: {}%'.format(acc*100))


Em fold #1: o melhor n_feature é: 10 e n_tree: 400
Acurácia do Fold = 85.98726114649682%
Em fold #2: o melhor n_feature é: 25 e n_tree: 100
Acurácia do Fold = 93.31210191082803%
Em fold #3: o melhor n_feature é: 15 e n_tree: 200
Acurácia do Fold = 96.48562300319489%
Em fold #4: o melhor n_feature é: 10 e n_tree: 100
Acurácia do Fold = 96.1661341853035%
Em fold #5: o melhor n_feature é: 20 e n_tree: 100
Acurácia do Fold = 94.56869009584665%

Acurácia do Método: 93.30396206833397%


# Gradient Boosting Machine
### Para o GBM (ou XGB) teste para numero de arvores = 30, 70, e 100, com learning rate de 0.1 e 0.05, e profundidade da arvore=5.

In [49]:
# acurácia inicial
acc = 0

# separa os dados transformados  std_secom em 5-fold
kf_5 = KFold(n = len(labels), n_folds=5)

#contador dos folds
count = 0

# para cada fold externo encontrar o melhor hiperparâmetro
for train, test in kf_5:
    count += 1
    kf_3 = KFold(n = len(labels[train]), n_folds=3)
    
    # faz um grid search do melhor k em cada conjunto de treino do 5-fold
    best_learning_rate, best_n_tree, best_acc = 0.1,0,0
        
    for ln in [0.1, 0.05]:
        for n_trees in [30, 70, 100]:
            new_acc = 0
            for train_hp, test_hp in kf_3:
                # define o GBM classificador 
                gbm = GBM(n_estimators=n_trees, learning_rate=ln, max_depth=5)
                # treina std_secom
                rf.fit(std_secom[train][train_hp], labels[train][train_hp]) 
                # avalia a acurária 
                new_acc += rf.score(std_secom[train][test_hp], \
                                     labels[train][test_hp])
        
            # acurácia média do hiperparâmetro k
            new_acc = new_acc/3
        
            # verifica se acurácia do hiperparametro k atual é a melhor
            if (new_acc > best_acc):
                best_acc        = new_acc
                best_learning_rate = ln
                best_n_tree    = n_trees
    
    # imprime a melhor acurácia obtida pelo fold interno
    print('Em fold #%d: o melhor learning rate é: %0.2f e n_tree: %d'
           % (count, best_learning_rate, best_n_tree))
    
    # ao fim do loop, calcula a acurácia para o melhor c e gamma
    gbm_out = GBM(n_estimators=best_n_tree, learning_rate=best_learning_rate)
    
    # treina com os valores do 5-fold externo
    gbm_out.fit(std_secom[train], labels[train]) 
    
    # obtém a acurácia usando o conjunto de teste do 5-fold externo
    print('Acurácia do Fold = {}%'.format(100*gbm_out.score(std_secom[test],\
           labels[test])))
   
    # acha a acurácia média do método
    acc += gbm_out.score(std_secom[test], labels[test])

acc /= 5    
print('\nAcurácia do Método: {}%'.format(acc*100))


Em fold #1: o melhor learning rate é: 0.10 e n_tree: 100
Acurácia do Fold = 71.97452229299363%
Em fold #2: o melhor learning rate é: 0.10 e n_tree: 70
Acurácia do Fold = 92.35668789808918%
Em fold #3: o melhor learning rate é: 0.10 e n_tree: 70
Acurácia do Fold = 94.88817891373802%
Em fold #4: o melhor learning rate é: 0.05 e n_tree: 30
Acurácia do Fold = 96.48562300319489%
Em fold #5: o melhor learning rate é: 0.10 e n_tree: 70
Acurácia do Fold = 93.9297124600639%

Acurácia do Método: 89.92694491361594%
